<a href="https://colab.research.google.com/github/Godiya-PT/Disater_related_Tweet_Detection/blob/main/Disaster_related_tweet_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code for Disater related tweets prediction

trial for Disaster related tweet prediction

Import libraries

In [25]:
# !pip install tensorflow_datasets
# !pip install NLP-python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
#from mpl_toolkits import mplot3d
# import spacy
# import re
# import nltk
# import seaborn as sns

# import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorflow import keras
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.preprocessing import LabelEncoder
# from collections import defaultdict
# from collections import Counter

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn import model_selection, naive_bayes
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.decomposition import TruncatedSVD
# from sklearn.preprocessing import MaxAbsScaler
# from scipy import stats
# from sklearn.metrics import make_scorer, roc_auc_score
# from scipy.linalg import svd
# from numpy import diag
# from scipy.sparse import csr_matrix

# from numpy import zeros
# from sklearn import svm

# #from nlp_functions import *

Import dataset

In [11]:
ds_train=pd.read_csv("/train.csv")
ds_test=pd.read_csv("/test.csv")
print("Train and Test data sets are imported successfully")

Train and Test data sets are imported successfully


Checking number of records in train data set and few records from train data set

In [12]:
print("Number of records in Train data set",len(ds_train.index))
ds_train.head()

Number of records in Train data set 7613


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Distinct keywords in train dataset

In [14]:
#Distinct keywords in train dataset
dist_keyword=ds_train['keyword'].value_counts()
print(dist_keyword)

fatalities               45
armageddon               42
deluge                   42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64


Visualize the keywords

In [26]:

fig = px.scatter(dist_keyword, x=dist_keyword.values, y=dist_keyword.index,size=dist_keyword.values)
fig.show()